# Introduction to Simulating Markov Chains
### Matias Bayas-Erazo

This notebook lays out a simulation of a very simple finite state Markov Chain. The transition probabilities can be found here:
https://lectures.quantecon.org/_images/nfs_ex1.png

We will simulate a sequence of states according to these transition probabilities first using a pure Python implementation and then Numbas and Cython to compare the speeds between the different implementations. 

#### 1. Pyhton Implementation

For the Python implementation, as a first pass I define a function `markov1` that takes as parameters the desired length of the sequence and the transition probabilities. The code below is more about clarity than efficiency. 

In [52]:
import numpy as np

def markov1(n,h_h,l_l):
    
    # States:
    states = [1, 0]  # 1 for high, 0 for low
    
    # Initial state
    init_state = states[0]
    
    # Empty array to store sequence of states:
    seq_states = np.empty(n)
    seq_states[0] = init_state
    x = np.random.uniform(0,1, size = n)

    # Main loop
    i = 0 
    while i < n-1:
        if seq_states[i] == 1:
            if x[i] <= h_h:
                seq_states[i+1] = states[0]
            else:
                seq_states[i+1] = states[1]         
        else:
            if x[i] <= l_l:
                seq_states[i+1] = states[1]
            else:
                seq_states[i+1] = states[0] 
        i += 1
    return(seq_states)


To make the code above more efficient, we proceed as follows:
    

In [53]:
def markov(n,h_h,l_l):
    # Initial state
    init_state = 1
    
    # Empty array to store sequence of states:
    seq_states = np.empty(n)
    seq_states[0] = init_state
    x = np.random.uniform(0,1, size = n)

    # Main loop
    i = 0 
    while i < n-1:
        if seq_states[i] == 1:
            seq_states[i+1] = x[i] < h_h     
        else:
            seq_states[i+1] = x[i] > l_l
        i += 1
    return(seq_states)


#### 2. Numba

Recall that we can speed up our functions created in Python using the package `numba` that automatically compiles the function for us. We import from `numba` the `jit` method and then simply pass our function to it:

In [54]:
from numba import jit
markov_numba = jit(markov)

Recall that if we don't want a different name for our function, we can simply compile the original one using the `@jit` decorator notation on top of our function.

#### 3. Cython

Finally, for our Cython implementation, we proceed as follows. First, we load the Cython extension in a notebook cell:

In [25]:
%load_ext Cython

Now, we can write our Cython code by beginning the cell with `%%cython` which tells the computer we are writing Cyhton code:

In [55]:
%%cython
import numpy as np
from numpy cimport int_t, float_t

def markov_cython(int n, double h_h, double l_l):

    # First, create numpy arrays for storing the sequence of states and the random draws.
    states_np_array = np.empty(n, dtype = int)
    x_np = np.random.uniform(0,1, size = n)

    # Create memoryviews of the arrays
    cdef int_t [:] states = states_np_array
    cdef float_t [:] x = x_np
    
    # Initial state
    states[0] = 1
    
    # Main Loop
    cdef int i = 0 
    while i < n-1:
        if states[i] == 1:
            states[i+1] = x[i] < h_h     
        else:
            states[i+1] = x[i] > l_l
        i += 1
    return np.asarray(states)
    

#### 4. Speed Comparison
Now, that we have the three versions we can compare the speeds and determine which one performs better:

In [56]:
%timeit markov(100000,0.8,0.9)

10 loops, best of 3: 102 ms per loop


In [58]:
%timeit markov_numba(100000,0.8,0.9)

1000 loops, best of 3: 1.17 ms per loop


In [59]:
%timeit markov_cython(100000,0.8,0.9)

1000 loops, best of 3: 1.72 ms per loop


Clearly, the fastest implementation is Numba. The Cython implementation is quite close but still falls behind. Either way, we can achieve significant improvements in speed by using Numba or programmin in Cython